## Documentation

To read more about the delete API, visit the [docs](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-delete.html).

![delete_documents_docs](../images/delete_documents_docs.png)

## Connect to ElasticSearch

In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

HOST = "http://localhost:9200"

es = Elasticsearch(HOST)
client_info = es.info()
print("Connected tp Elasticsearch!")
pprint(client_info.body)

Connected tp Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': 'IzAz_bJfQnS_zfMDjIPmJA',
 'name': 'eb6cd056e782',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-01-09T14:09:01.578835424Z',
             'build_flavor': 'default',
             'build_hash': '0f88dde84795b30ca0d2c0c4796643ec5938aeb5',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '8.11.3',
             'minimum_index_compatibility_version': '6.0.0-beta1',
             'minimum_wire_compatibility_version': '6.8.0',
             'number': '7.17.27'}}


/tmp/ipykernel_6244/13932774.py:7: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  client_info = es.info()


## Index documents

In [2]:
INDEX = "my_index"

settings = {
    "index": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    }
}

es.indices.delete(index=INDEX, ignore_unavailable=True)
es.indices.create(index=INDEX, settings=settings)

/tmp/ipykernel_6244/2757242129.py:10: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.delete(index=INDEX, ignore_unavailable=True)
/tmp/ipykernel_6244/2757242129.py:11: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  es.indices.create(index=INDEX, settings=settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'my_index'})

We are using the `dummy_data.json` file to insert multiple documents in the index. We store the id of each document in the `document_ids` list.

In [3]:
import json
from tqdm import tqdm

document_ids = []
dummy_data = json.load(open("../data/dummy_data.json"))
for document in tqdm(dummy_data, total=len(dummy_data)):
    response = es.index(index=INDEX, body=document)
    document_ids.append(response["_id"])

  0%|          | 0/3 [00:00<?, ?it/s]

/tmp/ipykernel_6244/2172172568.py:7: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.index(index=INDEX, body=document)
100%|██████████| 3/3 [00:00<00:00, 25.84it/s]


In [4]:
document_ids

['JngjJJUBpQvCJGK54k0m', 'J3gjJJUBpQvCJGK54k1y', 'KHgjJJUBpQvCJGK54k2G']

## Delete operation

This is an example of a successful operation. If the `ID` exists in the document, the `delete` operation won't return any errors.

In [5]:
response = es.delete(index=INDEX, id=document_ids[0])

/tmp/ipykernel_6244/1151246449.py:1: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  response = es.delete(index=INDEX, id=document_ids[0])


In [6]:
from pprint import pprint

pprint(response.body)

{'_id': 'JngjJJUBpQvCJGK54k0m',
 '_index': 'my_index',
 '_primary_term': 1,
 '_seq_no': 3,
 '_shards': {'failed': 0, 'successful': 1, 'total': 1},
 '_type': '_doc',
 '_version': 2,
 'result': 'deleted'}


This example shows that the `delete` operation fails when providing it with an `ID` that does not exist in the index.

In [7]:
try:
    response = es.delete(index=INDEX, id="id")
except Exception as e:
    print(e)

NotFoundError(404, "{'_index': 'my_index', '_type': '_doc', '_id': 'id', '_version': 1, 'result': 'not_found', '_shards': {'total': 1, 'successful': 1, 'failed': 0}, '_seq_no': 4, '_primary_term': 1}")
